In [ ]:
### Load Essential Packages for DICOM ---> PNG ###
import cv2
import pydicom
import numpy as np
from skimage import exposure
import os

from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
### Create New Directories for PNG Files ###
# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in os.walk('/media/bitlockermount/MESArthritis/'):
    path = root.split(os.sep)
    path2 = path[-1]
    path3 = ''.join(path2)
    os.makedirs('/home/john/Documents/MESArthritis/Recursive/' + path3, exist_ok=True)

In [ ]:
### Automatically Create PNG from DICOM for All Patients in Directory ###
import glob
from glob import iglob

for filename in glob.iglob('/media/bitlockermount/MESArthritis/**/*.dcm',recursive = True):
    ds = pydicom.read_file(filename) #read dicom image
    img = ds.pixel_array # get image array
    img[img > 1130] = 1130
    img[img <= 875] = 875
    img2 = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    img3 = cv2.resize(img2, (1200, 1200), interpolation = cv2.INTER_NEAREST)
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    path3 = os.path.join('/home/john/Documents/MESArthritis/Recursive/' + path2 + '/' + filenames.replace('.dcm','.png'))
    cv2.imwrite(path3,img3) # write png image

In [ ]:
### Automatically Create PNG from DICOM for Individual Patients in Single Subfolder ###
inputdir = '/media/bitlockermount/MESArthritis/MESA3010090/'
outdir = '/home/john/Documents/MESArthritis/MESA3010090/'

test_list = [ f for f in  os.listdir(inputdir)]

for f in test_list:   # remove "[:10]" to convert all images 
    ds = pydicom.read_file(inputdir + f) # read dicom image
    img = ds.pixel_array # get image array
    img[img > 1130] = 1130
    img[img <= 875] = 875   
    img2 = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    img3 = cv2.resize(img2, (1200, 1200), interpolation = cv2.INTER_NEAREST) 
    cv2.imwrite(outdir + f.replace('.dcm','.png'),img3) # write png image

In [ ]:
### Load Essential Packages for Automatic Selection of PNG Directly Above Aortic Arch ###
import numpy as np
import pandas as pd
import cv2
from imutils import paths
import argparse
import pickle
import vptree
import matplotlib.pyplot as plt
import time

In [ ]:
### Create Image Hashing ###
def dhash(image, hashSize=18):
    resized = cv2.resize(image, (hashSize + 1, hashSize))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

### Converting Data Types ###
def convert_hash(h):
    return int(np.array(h, dtype="float64"))

def hamming(a, b):
    return bin(int(a) ^ int(b)).count("1")

In [ ]:
imagePaths = list(paths.list_images("/home/john/Documents/MESArthritis/Recursive/MESA3010007/"))
hashes= {}

### Loading and Hashing the Images ###
for (i, img_path) in enumerate(imagePaths):
    img = cv2.imread(img_path)
    h = dhash(img)
    h = convert_hash(h)
    l = hashes.get(h, [])
    l.append(img_path)
    hashes[h] = l

In [ ]:
points = list(hashes.keys())
tree = vptree.VPTree(points, hamming) #Creating VP-Tree

In [ ]:
### Serializing VP-Tree ###
f = open("tree.pickle", "wb")
f.write(pickle.dumps(tree))
f.close()

### Serialize the Hashes to Dictionary ###
f = open("hashes.pickle", "wb")
f.write(pickle.dumps(hashes))
f.close()

In [ ]:
tree = pickle.loads(open("tree.pickle", "rb").read())
hashes = pickle.loads(open("hashes.pickle", "rb").read())

In [ ]:
imaged = cv2.imread("/home/john/Documents/MESArthritis/IM-0353-0076.png")

In [ ]:
### Compute the Hash for the Query Image, then Convert It ###
queryHash = dhash(imaged)
queryHash = convert_hash(queryHash)

In [ ]:
start = time.time()
#results = tree.get_all_in_range(queryHash, 20)
results = tree.get_n_nearest_neighbors(queryHash, 20)
results = sorted(results)
end = time.time()
print(">>> search took {} seconds".format(end - start))

In [ ]:
### Loop Over the Results and Select the Most Similar PNG ###
for (d, h) in results[:1]:
    resultPaths = hashes.get(h, [])
    print(">>> {} total image(s) with d: {}, h: {}".format(len(resultPaths), d, h))
    for resultPath in resultPaths:
        result = cv2.imread(resultPath)
#        cv2.imwrite('/home/john/Documents/MESArthritis/MESA3010058.png',result)
        plt.imshow(result)
        plt.show()

In [ ]:
import glob
import json
import os
import os.path as osp

import numpy as np
import PIL.Image
import cv2
import labelme
import numpy as np

DATA_DIR = '/home/john/Documents/MESArthritis/Aortic_Arch'
OUT_DIR = '/home/john/Documents/MESArthritis/Ground_Truth'
class_names = []
class_name_to_id = {}
for i, line in enumerate(open(DATA_DIR + '/labels.txt').readlines()):
    class_id = i - 1  # starts with -1
    class_name = line.strip()
    class_name_to_id[class_name] = class_id
    if class_id == -1:
        assert class_name == '__ignore__'
        continue
    elif class_id == 0:
        assert class_name == '_background_'
    class_names.append(class_name)
class_names = tuple(class_names)
print('class_names:', class_names)
out_class_names_file = osp.join(DATA_DIR, 'class_names.txt')
with open(out_class_names_file, 'w') as f:
    f.writelines('\n'.join(class_names))
print('Saved class_names:', out_class_names_file)

if osp.exists(OUT_DIR):
    print('Output directory already exists:', OUT_DIR)
    quit(1)
os.makedirs(OUT_DIR, exist_ok=True)
for label_file in sorted(glob.glob(osp.join(DATA_DIR, '*.json'))):
    with open(label_file) as f:
        base = osp.splitext(osp.basename(label_file))[0]
        data = json.load(f)
        img_file = osp.join(osp.dirname(label_file), data['imagePath'])
        img = np.asarray(PIL.Image.open(img_file))
        lbl, _ = labelme.utils.shapes_to_label(
            img_shape=img.shape,
            shapes=data['shapes'],
            label_name_to_value=class_name_to_id,
        )
        instance1 = np.copy(lbl)
        instance3 = np.copy(lbl)
        pos_1 = np.where(lbl==0)
        pos_3 = np.where(lbl==3)
        instance1[pos_1] = 0
        instance3[pos_3] = 0

        instance1 = instance1*255
        instance3 = instance3*255
        instance2 = np.subtract(instance1, instance3)
        os.makedirs(osp.join(OUT_DIR, base), exist_ok=True)
        os.makedirs(osp.join(OUT_DIR, base,'images'), exist_ok=True)
        PIL.Image.fromarray(img).save(osp.join(OUT_DIR, base, 'images', base + '.png'))
        os.makedirs(osp.join(OUT_DIR, base, 'masks'), exist_ok=True)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_instance1' + '.png'), instance1)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_instance2' + '.png'), instance2)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_instance3' + '.png'), instance3)

In [ ]:
import pixellib
from pixellib.custom_train import instance_custom_training

vis_img = instance_custom_training()
vis_img.load_dataset("/home/john/Documents/MESArthritis/ChestCT/")
vis_img.visualize_sample()

In [ ]:
import pixellib
from pixellib.custom_train import instance_custom_training

train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet101", num_classes= 3, batch_size = 12)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/ChestCT")
train_maskrcnn.train_model(num_epochs = 1, augmentation=False,  path_trained_models = "/home/john/Documents/MESArthritis/")

In [ ]:
import pixellib
from pixellib.custom_train import instance_custom_training

train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet101", num_classes= 3, batch_size = 12)
train_maskrcnn.load_pretrained_model("/home/john/Documents/MESArthritis/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("/home/john/Documents/MESArthritis/ChestCT")
train_maskrcnn.evaluate_model("/home/john/Documents/MESArthritis/Run3_92.h5")

In [ ]:
from pixellib.instance import custom_segmentation

segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 3, class_names= ["BG", "PMR", "PML", "SAT"], detection_threshold = 0.8, network_backbone = "resnet101")
segment_image.load_model("/home/john/Documents/MESArthritis/Run3_92.h5")

In [ ]:
import numpy as np
from PIL import Image
import cv2
import os

inputdir = '/home/john/Documents/MESArthritis/Aortic_Arch/'
outdir = '/home/john/Documents/MESArthritis/Inference/Output/'
outdir1 = '/home/john/Documents/MESArthritis/Inference/PM/PMR_Mask_'
outdir2 = '/home/john/Documents/MESArthritis/Inference/PM/PMR_'
outdir3 = '/home/john/Documents/MESArthritis/Inference/SAT/SAT_Mask_'
outdir4 = '/home/john/Documents/MESArthritis/Inference/SAT/SAT_'
outdir5 = '/home/john/Documents/MESArthritis/Inference/PM/PML_Mask_'
outdir6 = '/home/john/Documents/MESArthritis/Inference/PM/PML_'

test_list = [ f for f in  os.listdir(inputdir)]

for f in test_list:
    if f.endswith('.png'):
        segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
        a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]

        cv2.imwrite(outdir + f, output)
        a1 = np.array(a)
        b1 = np.array(b)
        c1 = np.array(c)
        for i in a1.tolist():
            mask1 = np.asarray(Image.open(inputdir + f))
            img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
            maskB = Image.fromarray(img1)
        for i in a1.tolist():
            mask2 = np.asarray(Image.open(inputdir + f))
            img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
            maskW = Image.fromarray(img2)
            ROI_PMR_Mask = np.subtract(maskW, maskB)
            ROI_PMR = np.subtract(maskW, np.asarray(Image.open(inputdir + f)))
            cv2.imwrite(outdir1 + f, ROI_PMR_Mask)
            cv2.imwrite(outdir2 + f, ROI_PMR)
        for i in b1.tolist():
            mask3 = np.asarray(Image.open(inputdir + f))
            img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
            maskB2 = Image.fromarray(img3)
        for i in b1.tolist():
            mask4 = np.asarray(Image.open(inputdir + f))
            img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
            maskW2 = Image.fromarray(img4)
            ROI_SAT_Mask = np.subtract(maskW2, maskB2)
            ROI_SAT = np.subtract(maskW2, np.asarray(Image.open(inputdir + f)))
            cv2.imwrite(outdir3 + f, ROI_SAT_Mask)
            cv2.imwrite(outdir4 + f, ROI_SAT)
        for i in c1.tolist():
            mask5 = np.asarray(Image.open(inputdir + f))
            img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
            maskB3 = Image.fromarray(img5)
        for i in c1.tolist():
            mask6 = np.asarray(Image.open(inputdir + f))
            img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
            maskW3 = Image.fromarray(img6)
            ROI_PML_Mask = np.subtract(maskW3, maskB3)
            ROI_PML = np.subtract(maskW3, np.asarray(Image.open(inputdir + f)))
            cv2.imwrite(outdir5 + f, ROI_PML_Mask)
            cv2.imwrite(outdir6 + f, ROI_PML)

In [ ]:
intersection = np.logical_and(PM_R, PM_L)
union = np.logical_or(PM_R, PM_L)
iou_score = np.sum(intersection) / np.sum(union)